In [2]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import string                              
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer 

stopwords_english = stopwords.words('english')

tokenizer = TweetTokenizer(preserve_case = False, 
                           strip_handles = True,
                           reduce_len    = True)
stemmer = PorterStemmer()

webs    = pd.ExcelFile("Webs.xls")
df      = webs.parse("Categorias")
textos = pd.DataFrame(columns=['Texto','Longitud' ,'Clase'])
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [x.text.strip() for x in html.findAll("p") if x.text != "\n"]
    for p in parrafos:
        if 15 < len(p.split()) < 150:
            texto_tokens = tokenizer.tokenize(p)
            texto_clean  = [ word for word in texto_tokens if
                            word not in stopwords_english and
                            word not in string.punctuation]
            texto_stem   = [stemmer.stem(word) for word in texto_clean]
            textos = textos.append({'Texto': texto_stem, 'Longitud': len(texto_stem),'Clase':df["CÓDIGO"][i]}, ignore_index=True)

In [3]:
textosBack = textos

In [4]:
textos

,Texto,Longitud,Clase
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",84,1
1,"[youth, alexand, tutor, aristotl, age, 16, phi...",79,1
2,"[alexand, endeavour, reach, end, world, great,...",60,1
3,"[alexand, born, pella, capit, kingdom, macedon...",51,1
4,"[sever, legend, surround, alexander', birth, c...",75,1
...,...,...,...
4900,"[mani, hunter-gather, conscious, manipul, land...",45,5
4901,"[agriculturalist, also, regularli, hunt, gathe...",54,5
4902,"[evid, suggest, big-gam, hunter-gather, cross,...",58,5
4903,"[hunter-gather, would, eventu, flourish, ameri...",72,5


In [5]:
textos.tail(500)

,Texto,Longitud,Clase
4405,"[neanderth, like, abl, surviv, similar, rang, ...",101,5
4406,"[nonetheless, oppos, bone, sewing-needl, stitc...",83,5
4407,"[sinc, human, neanderth, admixtur, known, occu...",32,5
4408,"[remain, middl, palaeolith, stone, tool, greek...",91,5
4409,"[given, high, trauma, rate, evid, heal, neande...",50,5
...,...,...,...
4900,"[mani, hunter-gather, conscious, manipul, land...",45,5
4901,"[agriculturalist, also, regularli, hunt, gathe...",54,5
4902,"[evid, suggest, big-gam, hunter-gather, cross,...",58,5
4903,"[hunter-gather, would, eventu, flourish, ameri...",72,5


In [6]:
textos.describe()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


,Texto,Longitud,Clase
count,4905,4905,4905
unique,4904,99,5
top,"[war, last, mani, year, length, end, acced, te...",52,2
freq,2,114,1055


In [7]:
textos.groupby(by=["Clase"]).count()

,Texto,Longitud
Clase,,
1,974,974
2,1055,1055
3,968,968
4,885,885
5,1023,1023


In [8]:
textos =  textos.sort_values('Longitud',ascending = True)
textos.tail(500)

,Texto,Longitud,Clase
1490,"[histor, parallel, tradit, draw, inspir, middl...",79,2
2059,"[1297, king, dini, portug, took, person, inter...",79,3
4585,"[contrast, oldowan, tool, result, fortuit, pro...",79,5
4662,"[farmer, sproat, witzel, disput, find, point, ...",79,5
304,"[upon, land, north, africa, greet, local, berb...",79,1
...,...,...,...
730,"[evid, import, indu, ur, found, around, 2350, ...",103,1
4526,"[late, ice-ag, climat, chang, caus, plant, com...",103,5
4123,"[human, speci, eventu, develop, much, larger, ...",105,5
1961,"[refer, name, articles.latimes.com, http, arti...",200,2


In [9]:
textos = textos.drop(textos[textos['Longitud'] < 20].index)
textos.groupby(by=["Clase"]).count()

,Texto,Longitud
Clase,,
1,920,920
2,981,981
3,908,908
4,854,854
5,941,941


In [9]:
# textos.to_csv("Historia.csv")

In [10]:
textos = textos.sort_index()

In [11]:
textos.head(100)

,Texto,Longitud,Clase
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",84,1
1,"[youth, alexand, tutor, aristotl, age, 16, phi...",79,1
2,"[alexand, endeavour, reach, end, world, great,...",60,1
3,"[alexand, born, pella, capit, kingdom, macedon...",51,1
4,"[sever, legend, surround, alexander', birth, c...",75,1
...,...,...,...
102,"[alexand, featur, promin, modern, greek, folkl...",69,1
103,"[pre-islam, middl, persian, zoroastrian, liter...",68,1
104,"[figur, dhul-qarnayn, liter, two-horn, one, me...",35,1
105,"[syriac, version, alexand, romanc, portray, id...",36,1


In [12]:
textos20 = pd.DataFrame(columns=['Texto','Longitud' ,'Clase'])
for i in textos.index:    
    largo = textos.loc[i,'Longitud']    
    # print(i)    
    # print(largo)
    partes = largo // 20
    indice = 0
    texto = textos.loc[i,'Texto']
    for k in range(partes):
        texto20 = []
        for j in range(20):
             texto20.append(texto[20 * k + j])
        textos20 = textos20.append({'Texto': texto20, 'Longitud': len(texto20),'Clase':textos["Clase"][i]}, ignore_index=True)        

In [13]:
textos

,Texto,Longitud,Clase
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",84,1
1,"[youth, alexand, tutor, aristotl, age, 16, phi...",79,1
2,"[alexand, endeavour, reach, end, world, great,...",60,1
3,"[alexand, born, pella, capit, kingdom, macedon...",51,1
4,"[sever, legend, surround, alexander', birth, c...",75,1
...,...,...,...
4900,"[mani, hunter-gather, conscious, manipul, land...",45,5
4901,"[agriculturalist, also, regularli, hunt, gathe...",54,5
4902,"[evid, suggest, big-gam, hunter-gather, cross,...",58,5
4903,"[hunter-gather, would, eventu, flourish, ameri...",72,5


In [14]:
textos20

,Texto,Longitud,Clase
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",20,1
1,"[bc, commonli, known, alexand, great, greek, ὁ...",20,1
2,"[pella, 356, bc, succeed, father, philip, ii, ...",20,1
3,"[age, thirti, creat, one, largest, empir, anci...",20,1
4,"[youth, alexand, tutor, aristotl, age, 16, phi...",20,1
...,...,...,...
10299,"[south, chile, mont, verd, citat, need, americ...",20,5
10300,"[common, style, stone, tool, product, make, kn...",20,5
10301,"[archaic, period, america, saw, chang, environ...",20,5
10302,"[mobil, hunter-gather, individu, group, start,...",20,5


In [16]:
# textos20.to_csv("Historia20.csv")

In [15]:
textos20.groupby(by=["Clase"]).count()

,Texto,Longitud
Clase,,
1,2097,2097
2,2094,2094
3,2039,2039
4,2027,2027
5,2047,2047


In [16]:
textos20.describe()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


,Texto,Longitud,Clase
count,10304,10304,10304
unique,10300,1,5
top,"[coordin, mw-parser-output, geo-default, mw-pa...",20,1
freq,3,10304,2097


In [17]:
textosLargo = textos.drop("Texto", axis = 1)
textosLargo

,Longitud,Clase
0,84,1
1,79,1
2,60,1
3,51,1
4,75,1
...,...,...
4900,45,5
4901,54,5
4902,58,5
4903,72,5


In [18]:
textosLargo.describe()

,Longitud,Clase
count,4604,4604
unique,87,5
top,52,2
freq,114,981


In [21]:
textos.groupby(by=["Longitud"]).count().head(50)

,Texto,Clase
Longitud,,
20,35,35
21,40,40
22,40,40
23,44,44
24,39,39
25,46,46
26,65,65
27,52,52
28,74,74


In [19]:
textos["partesde20"] = textos["Longitud"] // 20

In [20]:
textos.sample(10)

,Texto,Longitud,Clase,partesde20
1410,"[charlemagne', capitulari, quot, pope, benedic...",21,2,1
3847,"[govern, offici, respons, public, order, worri...",34,4,1
3371,"[napoleon', mistress, children, joséphin, prod...",51,4,2
448,"[punish, minor, crime, involv, either, imposit...",77,1,3
971,"[druid, fulfil, varieti, role, celtic, religio...",34,1,1
1863,"[macedonian, emperor, citi, constantinopl, flo...",49,2,2
2810,"[emperor, joseph, die, april, 1711, archduk, c...",61,3,3
4085,"[aegean, bronz, age, began, around, 3200, bc, ...",43,5,2
713,"[sumerian, agricultur, depend, heavili, irrig,...",45,1,2
3998,"[pebbl, tool, found, latest, first, southern, ...",64,5,3


In [21]:
textos["partesde20"].sum()

10304

In [22]:
textos.groupby(by=["partesde20"]).count()

,Texto,Longitud,Clase
partesde20,,,
1,1163,1163,1163
2,1696,1696,1696
3,1258,1258,1258
4,471,471,471
5,14,14,14
10,1,1,1
11,1,1,1
